<a href="https://colab.research.google.com/github/Med-Rokaimi/SDE-LSTM-Fin/blob/main/LSTM_LEVY_calibrated_MPA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#LSTM - Levy model




*   The levy params are calibrated using the Marine Predators Algorithm (MPA)- metaheuristic optimiser
*   Italy 40 index dataset

*   Best solution found by MPA: {'sigma': 0.10761706439322324, 'lam': 0.30660394101146315, 'm': 1.2272103632211795, 'v': 0.4221247439535006, 'r': 2.6708324043804357}

*  Best mse achieved : 0.0000040






In [ ]:

import warnings
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
from utils.pytorchtools import EarlyStopping

from config.args import Config
from meta_opts.MPA import MPA
from utils.helper import save
from data.data import create_dataset, pytorch_data_input
from utils.evaluation import  metric
from utils.helper import create_exp
warnings.filterwarnings("ignore")


###Levy solver

In [ ]:
# Merton jump path
def levy_solver(structure, T, steps, Npaths):
    r,m,v,lam,sigma =structure['r'], structure['m'], structure['v'], structure['lam'], structure['sigma']
    torch.manual_seed(4)
    r = torch.tensor(r)
    m = torch.tensor(m)
    v = torch.tensor(v)
    size = (steps, Npaths)
    dt = T / steps

    rates = torch.rand(steps, Npaths)
    poisson = torch.poisson(rates)
    poi_rv = torch.mul(poisson, torch.normal(m, v).cumsum(dim=0))
    geo = torch.cumsum(((r - sigma ** 2 / 2 - lam * (m + v ** 2 * 0.5)) * dt +
                        sigma * torch.sqrt(torch.tensor(dt)) * torch.normal(m,v)), dim=0)
    out = torch.exp(geo + poi_rv)

    return out


##LSTM Unit

In [ ]:
# LSTM model
class LSTM(nn.Module):

    def __init__(self, args, input_dim, structure):
        super(LSTM, self).__init__()

        self.args = args
        self.hidden_dim = args.hidden_units1
        self.input_dim = input_dim
        self.output_dim = args.pred_len
        self.layer_dim = args.num_layers
        # LSTM layers
        self.lstm = nn.LSTM(
            self.input_dim, self.hidden_dim, self.layer_dim, batch_first=True, bidirectional=True, dropout=args.dropout
        )

        # Fully connected layer
        self.fc_1 = nn.Linear(self.hidden_dim * 2, args.hidden_units2)  # fully connected
        self.fc_2 = nn.Linear(args.hidden_units2, self.output_dim)  # fully connected last layer
        self.relu = nn.ReLU()
        self.structure = structure


    def forward(self, x):
        sde_path = levy_solver(self.structure, self.output_dim, x.size(0), 1)

        h0 = torch.zeros(self.layer_dim * 2, x.size(0), self.hidden_dim, device=x.device).requires_grad_()
        # Initializing cell state for first input with zeros
        c0 = torch.zeros(self.layer_dim * 2, x.size(0), self.hidden_dim, device=x.device).requires_grad_()

        out, (hn, cn) = self.lstm(x, (h0, c0))
        out = out[:, -1, :]

        # Convert the final state to our desired output shape (batch_size, output_dim)
        out = self.fc_1(out)  # first dense
        out = self.relu(out)  # relu

        out = self.fc_2(out)  # final output
        out = out * sde_path

        return out

##Trainer class

In [ ]:

class TorchTrainer:
    def __init__(self, model, loss_fn, optimizer):

        self.model = model
        self.loss_fn = loss_fn
        self.optimizer = optimizer
        self.train_losses = []
        self.val_losses = []

    def train_step(self, x, y):

        self.model.train()
        # Makes predictions
        yhat = self.model(x)
        # Computes loss
        loss = self.loss_fn(y, yhat)
        # Computes gradients

        loss.backward()
        # Updates parameters and zeroes gradients
        self.optimizer.step()
        self.optimizer.zero_grad()
        # Returns the loss
        return loss.item()


    def train(self, train_loader, val_loader, batch_size, n_epochs, n_features, result_path, structure):
        best_loss = float('inf')
        early_stopping = EarlyStopping(patience=7, verbose=False)

        import time
        training_loss, validation_loss = [], []
        start_time = time.time()  # Record the start time
        for epoch in range(1, n_epochs + 1):
            self.optimizer.zero_grad()
            batch_losses = []
            for x_batch, y_batch in train_loader:
                x_batch = x_batch.view([batch_size, -1, n_features]).to(device)
                y_batch = y_batch.to(device)
                loss = self.train_step(x_batch, y_batch)
                batch_losses.append(loss)
            training_loss = np.mean(batch_losses)
            self.train_losses.append(training_loss)

            with torch.no_grad():
                batch_val_losses = []
                for x_val, y_val in val_loader:
                    x_val = x_val.view([batch_size, -1, n_features]).to(device)
                    y_val = y_val.to(device)
                    self.model.eval()
                    yhat = self.model(x_val)
                    val_loss = self.loss_fn(y_val, yhat).item()
                    batch_val_losses.append(val_loss)

                validation_loss = np.mean(batch_val_losses)
                self.val_losses.append(validation_loss)

            print(f'Epoch {epoch + 1}/{n_epochs}, Training Loss: {training_loss:.6f}, Validation Loss: {val_loss:.6f}, Best Loss : {best_loss:.6f}')
            if val_loss < best_loss:
                best_loss = val_loss
                saved_model_path = save(self.model, result_path, 'best', save_model=True)
                print(f'Best model saved with loss: {best_loss:.6f}')
            early_stopping(val_loss, self.model)
            if early_stopping.early_stop:
                print("Early stopping")

                break

        end_time = time.time()  # Record the end time
        runtime = end_time - start_time  # Calculate the runtime

        return saved_model_path, self.model, runtime


    def evaluate(self, test_loader, batch_size=1, n_features=2):

        with torch.no_grad():
            preds = []
            trues = []
            for x_test, y_test in test_loader:
                x_test = x_test.view([batch_size, -1, n_features]).to(device)
                y_test = y_test.to(device)
                self.model.eval()
                yhat = self.model(x_test)
                yhat=yhat.cpu().data.numpy()
                preds.append(yhat)
                y_test=y_test.cpu().data.numpy()
                trues.append(y_test)

        preds = np.array(preds)
        trues = np.array(trues)
        preds = preds.reshape(-1, preds.shape[-1] )
        trues = trues.reshape(-1, trues.shape[-1])
        return trues, preds, self.model




##the problem definition and objective function of the MPA

In [ ]:
def decode_solution(solution, encod_data):
    sigma = solution[0]
    lam = solution[1]
    r = solution[2]
    m = solution[3]
    v = solution[4]

    return {
        "sigma": sigma,
        "lam": lam,
        "m": m,
        "v": v,
        "r": r
    }

def obj_function(solution, result_path, encod_data):
    structure = decode_solution(solution, encod_data)

    model = LSTM(config, data['X_train'].shape[2], structure)
    optimizer = getattr(torch.optim, config.opt)(model.parameters(), lr=config.lr)

    loss_fn = nn.MSELoss(reduction="mean")

    ex = TorchTrainer(model, loss_fn=loss_fn, optimizer=optimizer)
    input_dim = len(features)

    #################################################
    # optimising
    #################################################

    saved_model_path, trained_model, runtime = ex.train(train_loader, val_loader, batch_size=config.batch_size,
                                                        n_epochs=config.epochs,
                                                        n_features=input_dim, result_path=result_path, structure = structure)
    best_model = trained_model
    trues, preds, model = ex.evaluate(
        test_loader_one,
        batch_size=1,
        n_features=input_dim)
    print("opt", ex)

    preds = preds[:, -1].reshape(-1, 1)
    trues = trues[:, -1].reshape(-1, 1)
    print(f" preds: {preds.shape}, trues {trues.shape}")
    metrics = metric(trues, preds)

    return metrics['mse'], trues, preds



In [ ]:
   # GWO solution bundries

LB = [0,0, 0, 0, 0]
UB = [4.99, 4.99, 4.99, 4.99, 4.99]

problem = {
        "fit_func": obj_function,
        "lb": LB,
        "ub": UB,
        "minmax": "min",
        "verbose": True,
        "save_population": False,
        "log_to": "console",
        "dataset": {}
    }


## the main function

In [ ]:

################################################
# Experimin SetUp
################################################

device = torch.device("cuda:0") if torch.cuda.is_available() else torch.device("cpu")
torch_seed, rs_seed = 2012, 4
torch.manual_seed(torch_seed)
rs = np.random.RandomState(rs_seed)

result_path = "./results"
EXCEL_EXP_PATH = "./results/exp.xlsx"
saved_model_path = ""
dataset_name = "Brent.csv"
dataset_path = 'dataset/' + dataset_name
target_column = 'Price'  # the column name of the target time series (brent or WTI)

model_decriptipn = 'LSTM + Merton Jump '
model_name = 'LSTM_Levy_Merton_calibrated by mpa'
save_model = True
config = Config(
    epochs=300,
    pred_len=1,
    seq_len=10,
    n_critic=1,
    model_name=model_name,
    dataset=target_column,
    lr=0.003054,
    num_layers=1,
    dropout=0.3,
    hidden_units1=8,
    hidden_units2=60,
    sde_parameters='N/A',
    batch_size=16,
    noise_type='normal',
    loss='MSELoss',
    opt='Adam',
    seeds={'torch_seed': torch_seed, 'rs_seed': rs_seed},
    sde='N/A'
)


#################################################
# Dataset
#################################################

# read the csv file
df = pd.read_csv(dataset_path)
#df = df[6:]

df = df[[target_column]]  # Price, WTI, SENT, GRACH
features = df.columns


train_size, valid_size, test_size =9019, 200,200 #3285 , 200, 200# 2040, 259, 65 #2000, 180, 200
data = create_dataset(df, target_column, train_size, valid_size, test_size, config.seq_len, config.pred_len)
train_loader, val_loader, test_loader , test_loader_one = pytorch_data_input(data, config.batch_size )
print(f"Data : {dataset_name}, {data['X_train'].shape} , {data['y_train'].shape}")
print()

#################################################
# create expermint instance
#################################################
jobID, ex_results_path = create_exp(result_path, 'exp.csv', config.model_name)
#################################################
# Build the model
#################################################


encod_data = {}
itr = 200
sol = MPA(obj_function, problem['lb'], problem['ub'], 5, 5, itr, 'obj_function', config, result_path,
           encod_data)

# copy solution returned from the optimiser algorithm
best_score = sol.best
best_solution = sol.bestIndividual
exe_time = sol.executionTime



In [ ]:
# decoding
best_sol = decode_solution(sol.Top_predator_pos, encod_data)
print("--------------------------\n")
print(f"Best solution: {best_sol}")
print(f"executionTime: {exe_time}")
mse = f"{sol.Top_predator_fit:.7f}"

print(f"Best score: {mse}")
print("--------------------------\n")

print(config)
print(df.columns)

--------------------------

Best solution: {'sigma': 0.10761706439322324, 'lam': 0.30660394101146315, 'm': 1.2272103632211795, 'v': 0.4221247439535006, 'r': 2.6708324043804357}
executionTime: 8606.27860879898
Best score: 0.0000040
--------------------------

Config(epochs=300, pred_len=1, seq_len=10, n_critic=1, model_name='LSTM_Levy_Merton_calibrated by mpa', dataset='Price', lr=0.003054, num_layers=1, dropout=0.3, hidden_units1=8, hidden_units2=60, sde_parameters='N/A', batch_size=16, noise_type='normal', opt='Adam', loss='MSELoss', seeds={'torch_seed': 2012, 'rs_seed': 4}, sde='N/A')
Index(['Price'], dtype='object')
